step 1- Load libraries

In [1]:
# ===== Step 1: Imports, seeds, and GPU setup =====

# Standard libraries
import os, random                              # OS utilities and reproducibility
import numpy as np                             # Numerical arrays

# TensorFlow / Keras
import tensorflow as tf                        # Main DL framework

# Keras layers & utilities
from tensorflow.keras.layers import (Input, SeparableConv2D, DepthwiseConv2D, BatchNormalization,
                                     Activation, AveragePooling2D, Dropout, Dense, Add, Lambda,
                                     GlobalAveragePooling2D, GlobalAveragePooling1D,
                                     MultiHeadAttention, Reshape, LayerNormalization, Conv2D)
from tensorflow.keras.models import Model      # Functional model API
from tensorflow.keras.optimizers import Adam   # Optimizer
from tensorflow.keras.losses import BinaryCrossentropy  # Binary classification loss
from tensorflow.keras.metrics import Accuracy  # Accuracy metric
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint  # Training callbacks
from tensorflow.keras import backend as K      # Keras backend for static shape queries

# Scikit-learn utilities
from sklearn.model_selection import StratifiedKFold                  # Stratified 5-fold CV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score  # Metrics

# Plotting (optional)
import matplotlib.pyplot as plt               # For charts (optional)

# Reproducibility
SEED = 42                                     # Fixed seed
np.random.seed(SEED)                          # Seed NumPy
tf.random.set_seed(SEED)                      # Seed TensorFlow
random.seed(SEED)                             # Seed Python

# Optional: make TensorFlow gentle with GPU memory
gpus = tf.config.list_physical_devices('GPU') # Detect GPUs
print("TensorFlow:", tf.__version__)          # Show TF version
print("GPUs:", gpus)                          # Show detected GPUs
if gpus:                                      # If GPU(s) are present
    for g in gpus:                            # Iterate each GPU
        try:
            tf.config.experimental.set_memory_growth(g, True)  # Enable memory growth
        except Exception as e:
            print("GPU memory growth not set:", e)             # Warn if not possible


TensorFlow: 2.20.0
GPUs: []


Step 2- Load arrays

In [2]:
# ===== Step 2: Load the arrays in the given path (no normalization) =====

# Exact Windows paths you provided (raw strings to avoid backslash escapes)
eeg_path    = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data\final_eeg_pp.npy"
labels_path = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data\final_eeg_labels.npy"

print("Loading EEG data...")                 # Status print
X = np.load(eeg_path)                        # Load EEG data, expected shape (N, 14, 200)
y = np.load(labels_path)                     # Load labels, expected shape (N,) or (N,1)
print("Loaded.")                             # Confirm loaded

print("Raw X shape:", X.shape, "| dtype:", X.dtype)  # Inspect EEG tensor
print("Raw y shape:", y.shape, "| dtype:", y.dtype)  # Inspect labels

y = np.squeeze(y).astype(int)                # Ensure labels are 1D int array
print("y squeezed shape:", y.shape, "| unique labels:", np.unique(y))  # Show label distribution

# Sanity checks on shapes and labels
assert X.ndim == 3, f"Expected X to be (N,14,200). Got {X.shape}"      # Check rank
assert X.shape[1] == 14 and X.shape[2] == 200, "Expected 14 electrodes and 200 samples per segment."  # Check dims
assert set(np.unique(y)).issubset({0,1}), "Labels must be {0,1}."      # Binary check

# Add channel dimension for Keras Conv2D: (N, 14, 200, 1)
X = X[..., np.newaxis]                         # Append channel dim
print("X after adding channel dim:", X.shape)  # Verify final input shape


Loading EEG data...
Loaded.
Raw X shape: (4770, 14, 200) | dtype: float64
Raw y shape: (4770,) | dtype: int64
y squeezed shape: (4770,) | unique labels: [0 1]
X after adding channel dim: (4770, 14, 200, 1)


Step 3: Normalising Data

In [3]:
# ===== Step 3: Normalization helper (per-fold; compute stats on train only) =====

def normalize_per_fold(X_train, X_val, eps=1e-8):
    """
    Normalizes X_train and X_val using mean/std computed from X_train only.
    Broadcasting shape is (1, 14, 1, 1), i.e., per-electrode normalization.
    """
    # Step 1: Compute mean & std from the training set only
    train_mean = np.mean(X_train, axis=(0, 2, 3), keepdims=True)   # -> (1, 14, 1, 1)
    train_std  = np.std( X_train, axis=(0, 2, 3), keepdims=True)   # -> (1, 14, 1, 1)

    # Safety: avoid division by (near) zero
    train_std = np.where(train_std < eps, 1.0, train_std)

    # Step 2: Apply normalization using training mean & std
    X_train_norm = ((X_train - train_mean) / train_std).astype(np.float32)
    X_val_norm   = ((X_val   - train_mean) / train_std).astype(np.float32)

    print("Per-fold normalization done.",
          "train_mean shape:", train_mean.shape, "train_std shape:", train_std.shape)

    return X_train_norm, X_val_norm


Step 4: Prepare 5-fold CV

In [4]:
# ===== Step 4: Prepare the data to perform 5-fold cross validation =====

N_SPLITS = 5                                   # Number of CV folds
skf = StratifiedKFold(n_splits=N_SPLITS,       # Stratified (preserves class ratios)
                      shuffle=True,            # Shuffle before splitting
                      random_state=SEED)       # Reproducible splits

print(f"Prepared StratifiedKFold with {N_SPLITS} splits.")  # Status
print("Total samples:", X.shape[0])                         # Dataset size


Prepared StratifiedKFold with 5 splits.
Total samples: 4770


Step 5: Model, compile, callbacks

In [5]:
# ===== Cell 1: EEGNet (generalized to your data shape) =====
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, BatchNormalization,
    Activation, AveragePooling2D, Flatten, Dense, Dropout
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

def create_cnn_model(input_shape, dropout_rate=0.5, num_classes=1, lr=1e-3):
    """
    EEGNet-like CNN generalized for input_shape=(n_channels, n_times, 1).
    Uses a depthwise spatial kernel that matches n_channels automatically.
    """
    n_channels, n_times, _ = input_shape

    inputs = Input(shape=input_shape)

    # Temporal filtering
    # Keep a reasonable temporal kernel, pad='same' so it works for short/long segments.
    x = Conv2D(16, (1, min(64, max(3, n_times))), padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)

    # Spatial depthwise conv over electrodes (kernel height = n_channels)
    x = DepthwiseConv2D((n_channels, 1), depth_multiplier=2, padding='valid', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D((1, 4))(x)
    x = Dropout(dropout_rate)(x)

    # Separable conv (spatio-temporal refinement)
    # Keep time kernel modest; cap at 16.
    x = SeparableConv2D(16, (1, min(16, max(3, n_times // 10))), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D((1, 8))(x)
    x = Dropout(dropout_rate)(x)

    # Classifier head
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)  # binary classification

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def make_callbacks(fold_idx):
    """EarlyStopping + ReduceLROnPlateau + ModelCheckpoint(.h5)"""
    es = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    rlrop = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-6,
        verbose=1
    )
    ckpt = ModelCheckpoint(
        filepath=f"best_fold_{fold_idx}.h5",
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    return [es, rlrop, ckpt]


In [6]:
# ===== Cell 2: 5-fold CV with confusion matrix + metrics using EEGNet =====

import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

# Assumptions:
# - X has shape (n_samples, n_channels, n_times, 1)
# - y has shape (n_samples,) with binary labels {0,1}
# - normalize_per_fold(X_train, X_val) is defined elsewhere in your notebook

EPOCHS = 200
BATCH_SIZE = 32
N_SPLITS = 5
RANDOM_STATE = 42

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

per_fold_metrics = []
histories = []

fold = 0
for train_idx, val_idx in skf.split(X, y):
    fold += 1
    print("\n" + "="*90)
    print(f"Fold {fold} — Train/Val split")
    print("Train idx:", train_idx.shape, "Val idx:", val_idx.shape)

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    print("X_train:", X_train.shape, "| y_train:", y_train.shape)
    print("X_val  :", X_val.shape,   "| y_val  :", y_val.shape)

    # ----- Per-fold normalization (train-only stats) -----
    X_train, X_val = normalize_per_fold(X_train, X_val)

    # Sanity checks
    train_global_mean = float(np.mean(X_train))
    train_global_std  = float(np.std(X_train))
    val_global_mean   = float(np.mean(X_val))
    val_global_std    = float(np.std(X_val))
    # If electrodes dimension is axis=1 (n_channels), std over (samples, time, features)
    train_per_elec_std = np.std(X_train, axis=(0, 2, 3))

    print(
        "Normalization check — "
        f"Train(mean,std)=({train_global_mean:.4f}, {train_global_std:.4f}) | "
        f"Val(mean,std)=({val_global_mean:.4f}, {val_global_std:.4f}) | "
        f"Train per-electrode std range: [{train_per_elec_std.min():.3f}, {train_per_elec_std.max():.3f}]"
    )

    # ----- Build model with per-fold input shape -----
    input_shape = X_train.shape[1:]  # (n_channels, n_times, 1)
    model = create_cnn_model(input_shape=input_shape, dropout_rate=0.5, num_classes=1, lr=1e-3)
    print(model.summary(line_length=120))

    callbacks = make_callbacks(fold)

    print("Training with EarlyStopping...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=2
    )
    histories.append(history.history)

    print("Predicting on validation fold...")
    y_prob = model.predict(X_val, batch_size=BATCH_SIZE).ravel()
    y_pred = (y_prob >= 0.5).astype(int)

    # ----- Confusion Matrix & Metrics -----
    cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()

    print("\nFold", fold, "Confusion Matrix (rows=true, cols=pred) [0=Class0, 1=Class1]:")
    print(cm)
    print(f"TN={tn}, FP={fp}, FN={fn}, TP={tp}")

    acc  = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred, pos_label=1, zero_division=0)
    sens = recall_score(y_val, y_pred,    pos_label=1, zero_division=0)
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    f1   = f1_score(y_val, y_pred, pos_label=1, zero_division=0)

    print("\nFold", fold, "Metrics (threshold=0.5):")
    print(f"Accuracy    : {acc:.4f}")
    print(f"Precision   : {prec:.4f}  (class 1)")
    print(f"Sensitivity : {sens:.4f}  (Recall for class 1)")
    print(f"Specificity : {spec:.4f}  (for class 0)")
    print(f"F1-score    : {f1:.4f}")

    per_fold_metrics.append({
        "fold": fold, "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp),
        "accuracy": float(acc), "precision": float(prec),
        "sensitivity": float(sens), "specificity": float(spec), "f1": float(f1)
    })

# ----- Per-fold summary -----
print("\n=== Per-Fold Metrics Summary ===")
for m in per_fold_metrics:
    print(f"Fold {m['fold']}: "
          f"Acc={m['accuracy']:.4f}, "
          f"Prec={m['precision']:.4f}, "
          f"Sens={m['sensitivity']:.4f}, "
          f"Spec={m['specificity']:.4f}, "
          f"F1={m['f1']:.4f} | "
          f"TN={m['tn']} FP={m['fp']} FN={m['fn']} TP={m['tp']}")

# ----- Mean ± Std across folds -----
accs  = [m["accuracy"]    for m in per_fold_metrics]
precs = [m["precision"]   for m in per_fold_metrics]
senss = [m["sensitivity"] for m in per_fold_metrics]
specs = [m["specificity"] for m in per_fold_metrics]
f1s   = [m["f1"]          for m in per_fold_metrics]

print("\n=== Mean ± Std over", N_SPLITS, "folds ===")
print(f"Accuracy    : {np.mean(accs):.4f} ± {np.std(accs):.4f}")
print(f"Precision   : {np.mean(precs):.4f} ± {np.std(precs):.4f}")
print(f"Sensitivity : {np.mean(senss):.4f} ± {np.std(senss):.4f}")
print(f"Specificity : {np.mean(specs):.4f} ± {np.std(specs):.4f}")
print(f"F1-score    : {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")



Fold 1 — Train/Val split
Train idx: (3816,) Val idx: (954,)
X_train: (3816, 14, 200, 1) | y_train: (3816,)
X_val  : (954, 14, 200, 1) | y_val  : (954,)
Per-fold normalization done. train_mean shape: (1, 14, 1, 1) train_std shape: (1, 14, 1, 1)
Normalization check — Train(mean,std)=(-0.0000, 1.0000) | Val(mean,std)=(-0.0004, 1.0616) | Train per-electrode std range: [1.000, 1.000]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━
┃ Layer (type)                                        ┃ Output Shape                           ┃               Para
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━
│ input_layer (InputLayer)                            │ (None, 14, 200, 1)                     │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ conv2d (Conv2D)                                     │ (None, 14, 200, 16)                    │                 1,
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ batch_normalization (BatchNormalization)            │ (None, 14, 200, 16)                    │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ depthwise_conv2d (DepthwiseConv2D)                  │ (None, 1, 200, 32)                     │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ batch_normalization_1 (BatchNormalization)          │ (None, 1, 200, 32)                     │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ activation (Activation)                             │ (None, 1, 200, 32)                     │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ average_pooling2d (AveragePooling2D)                │ (None, 1, 50, 32)                      │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ dropout (Dropout)                                   │ (None, 1, 50, 32)                      │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ separable_conv2d (SeparableConv2D)                  │ (None, 1, 50, 16)                      │                 1,
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ batch_normalization_2 (BatchNormalization)          │ (None, 1, 50, 16)                      │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ activation_1 (Activation)                           │ (None, 1, 50, 16)                      │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ average_pooling2d_1 (AveragePooling2D)              │ (None, 1, 6, 16)                       │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ dropout_1 (Dropout)                                 │ (None, 1, 6, 16)                       │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ flatten (Flatten)                                   │ (None, 96)                             │                   
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ dense (Dense)                                       │ (None, 64)                             │                 6,
├─────────────────────────────────────────────────────┼────────────────────────────────────────┼───────────────────
│ dense_1 (Dense)                                     │ (None, 1)                              │                   
└─────────────────────────────────────────────────────┴─

 Total params: 9,025 (35.25 KB)

 Trainable params: 8,897 (34.75 KB)

...

120/120 - 4s - 29ms/step - accuracy: 0.9775 - loss: 0.0621 - val_accuracy: 0.9927 - val_loss: 0.0209 - learning_rate: 1.0000e-03
Epoch 15/200

Epoch 15: val_loss did not improve from 0.02095
120/120 - 3s - 28ms/step - accuracy: 0.9767 - loss: 0.0629 - val_accuracy: 0.9927 - val_loss: 0.0277 - learning_rate: 1.0000e-03
Epoch 16/200

Epoch 16: val_loss improved from 0.02095 to 0.01981, saving model to best_fold_5.h5


120/120 - 3s - 28ms/step - accuracy: 0.9817 - loss: 0.0556 - val_accuracy: 0.9927 - val_loss: 0.0198 - learning_rate: 1.0000e-03
Epoch 17/200

Epoch 17: val_loss did not improve from 0.01981
120/120 - 3s - 29ms/step - accuracy: 0.9840 - loss: 0.0462 - val_accuracy: 0.9927 - val_loss: 0.0215 - learning_rate: 1.0000e-03
Epoch 18/200

Epoch 18: val_loss did not improve from 0.01981
120/120 - 4s - 30ms/step - accuracy: 0.9869 - loss: 0.0391 - val_accuracy: 0.9906 - val_loss: 0.0280 - learning_rate: 1.0000e-03
Epoch 19/200

Epoch 19: val_loss did not improve from 0.01981
120/120 - 4s - 37ms/step - accuracy: 0.9885 - loss: 0.0377 - val_accuracy: 0.9948 - val_loss: 0.0214 - learning_rate: 1.0000e-03
Epoch 20/200

Epoch 20: val_loss improved from 0.01981 to 0.01784, saving model to best_fold_5.h5


120/120 - 4s - 31ms/step - accuracy: 0.9898 - loss: 0.0316 - val_accuracy: 0.9958 - val_loss: 0.0178 - learning_rate: 1.0000e-03
Epoch 21/200

Epoch 21: val_loss did not improve from 0.01784
120/120 - 4s - 29ms/step - accuracy: 0.9830 - loss: 0.0428 - val_accuracy: 0.9927 - val_loss: 0.0287 - learning_rate: 1.0000e-03
Epoch 22/200

Epoch 22: val_loss did not improve from 0.01784
120/120 - 4s - 30ms/step - accuracy: 0.9848 - loss: 0.0382 - val_accuracy: 0.9937 - val_loss: 0.0192 - learning_rate: 1.0000e-03
Epoch 23/200

Epoch 23: val_loss did not improve from 0.01784
120/120 - 4s - 30ms/step - accuracy: 0.9885 - loss: 0.0353 - val_accuracy: 0.9958 - val_loss: 0.0189 - learning_rate: 1.0000e-03
Epoch 24/200

Epoch 24: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 24: val_loss did not improve from 0.01784
120/120 - 3s - 29ms/step - accuracy: 0.9906 - loss: 0.0287 - val_accuracy: 0.9969 - val_loss: 0.0202 - learning_rate: 1.0000e-03
Epoch 25/200

Epoch 25: val_l

120/120 - 4s - 29ms/step - accuracy: 0.9937 - loss: 0.0169 - val_accuracy: 0.9958 - val_loss: 0.0172 - learning_rate: 5.0000e-04
Epoch 27/200

Epoch 27: val_loss did not improve from 0.01722
120/120 - 4s - 32ms/step - accuracy: 0.9950 - loss: 0.0183 - val_accuracy: 0.9958 - val_loss: 0.0225 - learning_rate: 5.0000e-04
Epoch 28/200

Epoch 28: val_loss did not improve from 0.01722
120/120 - 4s - 35ms/step - accuracy: 0.9934 - loss: 0.0220 - val_accuracy: 0.9958 - val_loss: 0.0216 - learning_rate: 5.0000e-04
Epoch 29/200

Epoch 29: val_loss did not improve from 0.01722
120/120 - 3s - 28ms/step - accuracy: 0.9948 - loss: 0.0159 - val_accuracy: 0.9969 - val_loss: 0.0187 - learning_rate: 5.0000e-04
Epoch 30/200

Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 30: val_loss did not improve from 0.01722
120/120 - 3s - 29ms/step - accuracy: 0.9929 - loss: 0.0201 - val_accuracy: 0.9958 - val_loss: 0.0229 - learning_rate: 5.0000e-04
Epoch 31/200

Epoch 31: val_l

120/120 - 4s - 30ms/step - accuracy: 0.9963 - loss: 0.0118 - val_accuracy: 0.9969 - val_loss: 0.0146 - learning_rate: 1.2500e-04
Epoch 37/200

Epoch 37: val_loss did not improve from 0.01457
120/120 - 4s - 29ms/step - accuracy: 0.9974 - loss: 0.0094 - val_accuracy: 0.9969 - val_loss: 0.0168 - learning_rate: 1.2500e-04
Epoch 38/200

Epoch 38: val_loss did not improve from 0.01457
120/120 - 3s - 29ms/step - accuracy: 0.9963 - loss: 0.0123 - val_accuracy: 0.9969 - val_loss: 0.0191 - learning_rate: 1.2500e-04
Epoch 39/200

Epoch 39: val_loss did not improve from 0.01457
120/120 - 3s - 28ms/step - accuracy: 0.9971 - loss: 0.0099 - val_accuracy: 0.9969 - val_loss: 0.0196 - learning_rate: 1.2500e-04
Epoch 40/200

Epoch 40: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 40: val_loss did not improve from 0.01457
120/120 - 4s - 30ms/step - accuracy: 0.9966 - loss: 0.0078 - val_accuracy: 0.9969 - val_loss: 0.0175 - learning_rate: 1.2500e-04
Epoch 41/200

Epoch 41: val_lo